# **Set according to environment (e.g. local, Google Colab...)**

In [14]:
project_folder = ''

# **Body**

In [15]:
from custom_libraries.miscellaneous import *
from custom_libraries.image_dataset import *
import numpy as np

In [16]:
import math

def init_weights(masker):

    density = np.count_nonzero(masker == 1) / masker.size
    stddev = math.sqrt(2/(masker.shape[0]*density))
    weights = np.random.normal(size=masker.shape, loc = 0., scale=stddev)
    weights[np.where(masker == 0)] = 0
    weights = np.reshape(weights, [-1])
    weights = weights[np.where(weights != 0)]
    return weights

class treeLayer(tf.keras.layers.Layer):

    def __init__(self, Input_size=3072, use_bias=False, divisor = 2):
        super(treeLayer, self).__init__()
        self.bias = None
        self.summer = None
        self.kernel = None
        self.Input_size = Input_size
        self.use_bias = use_bias
        self.divisor = divisor


    def build(self, input_shape):

        self.summer = np.zeros((self.Input_size, self.Input_size // self.divisor))
        for i in range(self.Input_size):
            self.summer[i, i // self.divisor] = 1

        initializer = init_weights(self.summer)
        initializer = tf.keras.initializers.Constant(initializer)

        self.kernel = self.add_weight(shape=(1, self.Input_size),
                                      initializer=initializer,
                                      trainable=True, dtype=tf.float32)

        self.summer = tf.convert_to_tensor(self.summer, dtype=tf.float32)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.Input_size//self.divisor,),
                                        initializer=tf.keras.initializers.Zeros,
                                        trainable=True)

    def call(self, inputs):
        #print(inputs.shape, self.kernel.shape)
        x = tf.math.multiply(inputs, self.kernel)
        x = tf.matmul(x, self.summer)
        #masked_weights = tf.multiply(self.kernel, self.summer)
        #x = tf.matmul(inputs, masked_weights)
        if self.use_bias:
            x = tf.nn.bias_add(x, self.bias)

        x = tf.nn.leaky_relu(x, alpha = .01)
        return x

def create_model(input_size, num_trees=1, use_bias=False):
    model = tf.keras.Sequential()
    while input_size > num_trees:

        # Check if image is 3-channel
        if input_size % 3 == 0:
            divisor = 3
        else:
            divisor = 2

        model.add(treeLayer(input_size, use_bias=use_bias, divisor=divisor))
        input_size = input_size // divisor

    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', use_bias=use_bias))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-08),
                  loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.AUTO),
                  metrics=['binary_crossentropy', 'acc'])

    return model


In [17]:
import gc

# Initialize settings
bs = 256
trials = 10
epochs = 2000
trees_set = [1]

# Load class-dataset list
# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

classes = [[3, 5, 'mnist'],
           [0, 6, 'fmnist'],
           [14, 17, 'emnist'],
           [2, 6, 'kmnist'],
           [3, 5, 'cifar10'],
           [5, 6, 'svhn'],
           [3, 5, 'usps']]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=60)

#history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
history = np.zeros((len(classes), trials, len(trees_set), 2))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    test_ds = ImageDataset(ds, 'test', data_dir=None)
    train_ds = ImageDataset(ds, 'train', data_dir=None)

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)

    for k, trees in enumerate(trees_set):

        print(f"{trees}-FCNN")
        X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                             validation_size=len(test_ds.images))
        X_test, y_test = test_ds.subset()
        test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(
            lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

        #if history[j, -1, k, 0] != 0:
        #  continue

        for i in range(trials):
            print(f"Trial {i + 1}")

            #with tf.device('/device:GPU:0'):

            model = create_model(input_size=X_train[i].shape[1] * trees, num_trees=trees, use_bias=False)

            train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)
            valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).map(
                lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

            fit_history = model.fit(x=train_set, batch_size=bs, epochs=epochs,
                                    validation_data=valid_set, validation_batch_size=bs,
                                    callbacks=[callback], verbose=0)
            print_fit_history(fit_history, epochs)

            evaluate_history = model.evaluate(x=test_set, batch_size=bs, verbose=0)
            print_evaluate_history(evaluate_history)

            history[j, i, k] = evaluate_history[1:]

            np.save(project_folder + 'results/ktree_history.npy', history,
                    allow_pickle=True)

            del model, train_set, valid_set
            gc.collect()

Dataset: mnist / Pair: 3-5
1-FCNN
Trial 1
Epochs: 695/2000 - Train BCE: 0.0655, accuracy: 99.07% - Validation BCE: 0.1721, accuracy: 95.81%
Test BCE: 0.213, accuracy: 94.16%
Trial 2
Epochs: 558/2000 - Train BCE: 0.1592, accuracy: 96.06% - Validation BCE: 0.2544, accuracy: 91.1%
Test BCE: 0.2417, accuracy: 92.22%
Trial 3
Epochs: 395/2000 - Train BCE: 0.1615, accuracy: 97.1% - Validation BCE: 0.4145, accuracy: 89.47%
Test BCE: 0.2627, accuracy: 92.48%
Trial 4
Epochs: 648/2000 - Train BCE: 0.2353, accuracy: 92.54% - Validation BCE: 0.3216, accuracy: 87.37%
Test BCE: 0.3012, accuracy: 89.48%
Trial 5
Epochs: 454/2000 - Train BCE: 0.6917, accuracy: 52.75% - Validation BCE: 0.6885, accuracy: 55.26%
Test BCE: 0.6912, accuracy: 53.1%
Trial 6
Epochs: 540/2000 - Train BCE: 0.177, accuracy: 94.82% - Validation BCE: 0.3274, accuracy: 88.95%
Test BCE: 0.2993, accuracy: 90.64%
Trial 7
Epochs: 63/2000 - Train BCE: 0.6908, accuracy: 53.47% - Validation BCE: 0.6947, accuracy: 50.0%
Test BCE: 0.6912, acc

In [18]:
history = np.load(project_folder + 'results/ktree_history.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{trees}-tree")
        print(f"Accuracy: {round(np.mean(history[j,:,k,1]), 4)} ± {round(np.std(history[j,:,k,1]), 4)}")

RESULTS:
Dataset: mnist / Pair: 3-5
1-tree
Accuracy: 0.7846 ± 0.1705
Dataset: fmnist / Pair: 0-6
1-tree
Accuracy: 0.704 ± 0.135
Dataset: emnist / Pair: 14-17
1-tree
Accuracy: 0.8271 ± 0.1361
Dataset: kmnist / Pair: 2-6
1-tree
Accuracy: 0.6985 ± 0.1306
Dataset: cifar10 / Pair: 3-5
1-tree
Accuracy: 0.5093 ± 0.0187
Dataset: svhn / Pair: 5-6
1-tree
Accuracy: 0.5549 ± 0.0362
Dataset: usps / Pair: 3-5
1-tree
Accuracy: 0.5724 ± 0.1265
